In [ ]:
import h5py
import matplotlib.pyplot as plt
import mpl_lego as mplego
import neuropacks as packs
import numpy as np
import os

from mpl_lego.ellipse import plot_cov_ellipse
from noise_correlations import analysis, utils 

In [ ]:
mplego.style.use_latex_style()

In [ ]:
base_path = '/storage/fits/neurocorr/exp09'

In [ ]:
pvc11_1_path = os.path.join(base_path, 'exp09_1_pvc11_15_1000_1000.h5')
pvc11_2_path = os.path.join(base_path, 'exp09_2_pvc11_15_1000_1000.h5')
pvc11_3_path = os.path.join(base_path, 'exp09_3_pvc11_15_1000_1000.h5')
ret2_path = os.path.join(base_path, 'exp09_ret2_15_1000_1000.h5')
ecog_path = os.path.join(base_path, 'exp09_ecog_15_3000_1000.h5')

In [ ]:
pvc11_1 = h5py.File(pvc11_1_path, 'r')
pvc11_2 = h5py.File(pvc11_2_path, 'r')
pvc11_3 = h5py.File(pvc11_3_path, 'r')
ret2 = h5py.File(ret2_path, 'r')
ecog = h5py.File(ecog_path, 'r')
results = [pvc11_1, pvc11_2, pvc11_3, ret2, ecog]
n_max_units = pvc11_1['units'].shape[2]
dims = 3 + np.arange(n_max_units - 2)

In [ ]:
percentile_groups = [
    'p_s_lfi',
    'p_r_lfi',
    'p_fa_lfi'
]
value_groups = [
    'v_s_lfi',
    'v_r_lfi',
    'v_fa_lfi'
]
colors = [
    'gray',
    'red',
    'fuchsia'
]
labels = mplego.labels.bold_text([
    'Shuffle',
    'Rotation',
    'FA'
])
v1_labels = mplego.labels.bold_text([
    'Dataset 1',
    'Dataset 2', 
    'Dataset 3'
])
titles = mplego.labels.bold_text(['V1', 'Retina', 'AC'])

In [ ]:
"""
Figure Settings
"""
# Spacing
wspace = 0.3
hspace = 0.35
title_pad = 20
# Line settings
linewidth = 2
vertical_linewidth = 4
linestyles = ['-', '--', ':']
# Fill settings
fill_alpha = 0.1
# Percentile bounds for curves
percentile_lower = 40
percentile_upper = 60
# Legend settings
legend_size = 15
handlelength = 0.3
handletextpad = 0.4
# Font sizes
label_font_size = 17
title_font_size = 23
tick_font_size = 15
subplot_label_size = 23
subplot_x = -0.16
subplot_y = 1.10
# Histogram settings
bins = 9

"""
Figure 4
"""
fig, axes = plt.subplots(2, 3, figsize=(15, 9))
plt.subplots_adjust(wspace=wspace, hspace=hspace)

"""
First Row: Example distributions
"""
dim_idx = 0
dimstim_idxs = [1484, 995, 313]

for idx, result in enumerate([pvc11_1, ret2, ecog]):
    dimstim_idx = dimstim_idxs[idx]
    lfi = result['v_lfi'][dim_idx, dimstim_idx]
    axes[0, idx].axvline(lfi, color='C0', linewidth=vertical_linewidth)

    for (v_group, p_group, color) in zip(value_groups, percentile_groups, colors):
        values = result[v_group][dim_idx, dimstim_idx]
        axes[0, idx].hist(
            values,
            color=color,
            histtype='step',
            linewidth=linewidth,
            bins=bins,
            label=f'$p={result[p_group][dim_idx, dimstim_idx]:0.2f}$')

axes[0, 0].legend(
    loc=2,
    handlelength=handlelength,
    handletextpad=handletextpad,
    prop={'size': legend_size})
axes[0, 1].legend(
    loc=1,
    handlelength=handlelength,
    handletextpad=handletextpad,
    prop={'size': legend_size})
axes[0, 2].legend(
    loc=2,
    handlelength=handlelength,
    handletextpad=handletextpad,
    prop={'size': legend_size})
        
# Set bounds
axes[0, 0].set_xlim([0.0006, 0.0014])
axes[0, 1].set_xlim([0.001, 0.0022])
axes[0, 2].set_xlim([0, 8])

axes[0, 0].set_xticks([0.0006, 0.0009, 0.0012])
axes[0, 1].set_xticks([0.0012, 0.0016, 0.0020])
axes[0, 2].set_xticks([0, 2, 4, 6, 8])

axes[0, 0].set_ylim([0, 350])
axes[0, 1].set_ylim([0, 350])
axes[0, 2].set_ylim([0, 300])

axes[0, 0].set_yticks([0, 100, 200, 300])
axes[0, 1].set_yticks([0, 100, 200, 300])
axes[0, 2].set_yticks([0, 100, 200, 300])


for (ax, title) in zip(axes[0], titles):
    ax.tick_params(labelsize=tick_font_size)

    ax.set_xlabel(r'\textbf{LFI}', fontsize=label_font_size)
    ax.set_ylabel(r'\textbf{Frequency}', fontsize=label_font_size)
    ax.set_title(title, fontsize=title_font_size, pad=title_pad)

"""
Second Row
"""
# Enumerate over results
for idx, result in enumerate(results):
    # V1 data is in same axis: check for this
    if idx == 0 or idx == 1 or idx == 2:
        ax = axes[1, 0]
        linestyle = linestyles[idx]
    else:
        ax = axes[1, idx - 2]
        linestyle = linestyles[0]

    # Iterate over null models
    for group, color in zip(percentile_groups, colors):
        # Calculate median and bounds for each null model
        percentiles = result[group][:]
        median = np.median(percentiles, axis=1)
        lower = np.percentile(percentiles, q=percentile_lower, axis=1)
        upper = np.percentile(percentiles, q=percentile_upper, axis=1)
        # Fill region between percentile bounds
        ax.fill_between(
            x=dims,
            y1=lower,
            y2=upper,
            color=color,
            alpha=fill_alpha,
            linestyle=linestyle)
        ax.plot(
            dims,
            median,
            linewidth=linewidth,
            color=color,
            linestyle=linestyle)

# Set bounds
axes[1, 0].set_ylim(bottom=1e-3)
axes[1, 1].set_ylim(bottom=1e-4)
axes[1, 2].set_ylim(bottom=1)

# Create legend outside of plot
for color, label in zip(colors, labels):
    axes[1, -1].plot([], [], color=color, label=label, linewidth=linewidth)
axes[1, -1].legend(
    loc='center left',
    bbox_to_anchor=(1.0, 0.5),
    prop={'size': legend_size})

# Set axis limits, scales, and labels
for ax in axes[1]:
    ax.set_xlim([3, 15])
    ax.set_ylim([0, 1])
    ax.set_xticks([3, 5, 10, 15])
    ax.tick_params(labelsize=tick_font_size)
    
    ax.set_xlabel(r'\textbf{Dimlet Dimension}', fontsize=label_font_size)
    ax.set_ylabel(r'\textbf{Percentile}', fontsize=label_font_size)
    
# Additional legend for the V1 curves
for idx, linestyle in enumerate(linestyles):
    axes[1, 0].plot(
        [], [],
        linewidth=2,
        linestyle=linestyle,
        color='black',
        label=r'\textbf{Dataset ' + f'{idx}' + '}')
axes[1, 0].legend(loc='best', prop={'size': 12})

# Apply subplot labels
mplego.labels.apply_subplot_labels(
    axes.ravel(),
    bold=True,
    x=subplot_x,
    y=subplot_y,
    size=subplot_label_size)

plt.savefig('figure4.pdf', bbox_inches='tight')
plt.show()